In [1]:
import rasterio as rs
import numpy as np
import matplotlib.pyplot as plt

import FCPGtools as cpg

In [2]:
upstreamFDRflesri = './data/NHDPlusV21_CO_14_14a_FdrFac_01/NHDPlusCO/NHDPlus14/NHDPlusFdrFac14a/fdr/hdr.adf'
downstreamFDRflesri = './data/NHDPlusV21_CO_15_15b_FdrFac_01/NHDPlusCO/NHDPlus15/NHDPlusFdrFac15b/fdr/hdr.adf'

upstreamFACfltau = './data/14a_fac.tif'
upstreamFDRfltau = './data/14a_fdr.tif'
downstreamFACfltau = './data/15b_fac.tif'
downstreamFDRfltau = './data/15b_fdr.tif'

In [3]:
cpg.tauDrainDir(upstreamFDRflesri, upstreamFDRfltau)
cpg.tauDrainDir(downstreamFDRflesri, downstreamFDRfltau)

Reclassifying Flow Directions...
TauDEM drainage direction written to: ./data/14a_fdr.tif
Reclassifying Flow Directions...
TauDEM drainage direction written to: ./data/15b_fdr.tif


In [4]:
cpg.tauFlowAccum(upstreamFDRfltau, upstreamFACfltau)

Accumulating Data...
mpiexec -bind-to rr -n 1 aread8 -p ./data/14a_fdr.tif -ad8 ./data/14a_fac.tif -nc


In [5]:
# do some work to create the update dictionary
x,y,d,w = cpg.findLastFACFD(upstreamFACfltau, upstreamFACfltau)
#facMax = cpg.queryPoint(x,y,upstreamFACfltau)
#dx,dy = cpg.FindDownstreamCellTauDir(d,x,y,w)
cpg.createUpdateDict(x,y,d,'14','./data/huc14.json')

Update dictionary found: ./data/huc14.json
Updating dictionary...


{'14': {'x': ['-1371809.9999999995'],
  'y': ['1647749.9999999963'],
  'maxUpstreamFAC': ['183857900.0'],
  'vars': ['maxUpstreamFAC']}}

In [6]:
cpg.makeFACweight(downstreamFDRfltau,'./data/downstream_test_FACweight.tif')

In [7]:
cpg.adjustFAC(downstreamFDRfltau, './data/downstream_test_FACweight.tif',
              './data/huc14.json',downstreamFDRfltau,'./data/ds_adjFAC.tif',8)

Updating FAC weighting grid with value from 14 FAC
Accumulating Data...
mpiexec -bind-to rr -n 8 aread8 -p ./data/15b_fdr.tif -ad8 ./data/ds_adjFAC.tif -wg ./data/downstream_test_FACweight.tif -nc
